# Advent of Code 2024 - J06

In [ ]:
def read_input(kind):
    assert kind in ('input', 'example'), '"kind" must be "input" or "example"'
    
    with open(kind) as f:
        for line in f:
            
    return 